# Basic neural network LSTM with Tensorflow

In [0]:
import numpy as np
from random import shuffle

import tensorflow as tf

import collections

In [0]:
# DATOS DE ENTRENAMIENTO DE ENTRADA

train_input = ['{0:020b}'.format(i) for i in range(2 ** 20)]
shuffle(train_input)
#print(train_input[:10])
train_input = [map(int, i) for i in train_input]  # lo convierte a int
ti = []
for i in train_input:
    temp_list = []
    for j in i:
        temp_list.append([j])
    ti.append((temp_list))
train_input = ti
print(train_input[:1000])

[[[1], [1], [1], [1], [1], [1], [1], [0], [1], [1], [1], [0], [0], [1], [1], [1], [0], [0], [1], [1]], [[1], [1], [0], [1], [0], [1], [1], [1], [0], [1], [0], [0], [1], [1], [0], [1], [0], [1], [0], [0]], [[1], [1], [0], [1], [1], [0], [0], [1], [1], [0], [1], [1], [1], [1], [0], [0], [0], [1], [1], [1]], [[0], [0], [1], [1], [0], [1], [0], [0], [0], [1], [0], [1], [1], [1], [0], [0], [0], [1], [0], [0]], [[0], [1], [1], [0], [1], [1], [1], [1], [0], [1], [0], [1], [1], [0], [0], [0], [1], [1], [1], [1]], [[1], [1], [0], [1], [0], [1], [0], [0], [1], [0], [1], [1], [1], [0], [1], [1], [1], [0], [0], [0]], [[0], [1], [0], [1], [1], [0], [0], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [1], [1], [0]], [[1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [1], [0], [0], [0], [0], [0], [1], [1]], [[0], [0], [1], [1], [1], [0], [1], [1], [0], [1], [1], [1], [0], [0], [0], [1], [1], [1], [0], [1]], [[0], [1], [1], [1], [0], [1], [0], [0], [0], [0], [1], [1], [0], [0], [1], [0], 

In [0]:
# DATOS DE ENTRENAMIENTO DE SALIDA

train_output = []
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count += 1
    temp_list = ([0] * 21)
    temp_list[count] = 1
    train_output.append(temp_list)
print(train_output[:10])

NameError: ignored

In [0]:
# SE DEJA EL 1% COMO DATOS DE PRUEBA DE LOS DATOS PARA EL ENTRENAMIENTO

NUM_EXAMPLES = 10000
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:]  # everything beyond 10,000
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES]  # till 10,000

print(test_input[:10])

[array([[1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0]]), array([[1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0]]), array([[1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0]]), array([[1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0]]), array([[0],
       [1],
       

In [0]:
# DEFINICION DE VARIABLES ENTRADA Y SALIDA

# marcadores de posición que serán suministrados con datos despues
data = tf.placeholder(tf.float32, [None, 20,1])  # las tres dimensiones descritas más arriba, el Batch Size se deja en desconocido para ser determinado en tiempo de ejecución
target = tf.placeholder(tf.float32, [None, 21])

In [0]:
# SE CREA LA CELDA DE LA RNN

# Primera parte: build the graph
num_hidden = 24  # numero de capas ocultas, un numero de capas muy altas puede producir overfitting
cell = tf.nn.rnn_cell.LSTMCell(num_hidden, state_is_tuple=True)

val, state = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)  # se pasa los datos de entrada a la red y se almace el val de salida

print(val.get_shape())

val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)

print(val.get_shape())
print(last.get_shape())

weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))

print(weight.get_shape())
print(bias.get_shape())

prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction, 1e-10, 1.0)))  # perdida de entropia cruzada, para calcular costo


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
(?, 20, 24)
(20, ?, 24)
(?, 24)
(24, 21)
(21,)


In [0]:
# optimizador que buscan el menor valor posible del cross_entropy
optimizer = tf.train.AdamOptimizer()
minimize = optimizer.minimize(cross_entropy)

# para tener una idea de la exactitud del modelo
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

# SE INICIALIZAN TODAS LAS VARIABLES Y SE CREA LA SESSIÓN
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
# ENTRENAMIENTO

batch_size = 1000
no_of_batches = int(len(train_input) / batch_size)
epoch = 200
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr + batch_size], train_output[ptr:ptr + batch_size]
        ptr += batch_size
        sess.run(minimize, {data: inp, target: out})
    print("Epoch - ", str(i))
incorrect = sess.run(error, {data: test_input, target: test_output})
print('Epoch {:2d} error {:3.1f}%'.format(i + 1, 100 * incorrect))


Epoch -  0
Epoch -  1
Epoch -  2
Epoch -  3
Epoch -  4
Epoch -  5
Epoch -  6
Epoch -  7
Epoch -  8
Epoch -  9
Epoch -  10
Epoch -  11
Epoch -  12
Epoch -  13
Epoch -  14
Epoch -  15
Epoch -  16
Epoch -  17
Epoch -  18
Epoch -  19
Epoch -  20
Epoch -  21
Epoch -  22
Epoch -  23
Epoch -  24
Epoch -  25
Epoch -  26
Epoch -  27
Epoch -  28
Epoch -  29
Epoch -  30
Epoch -  31
Epoch -  32
Epoch -  33
Epoch -  34
Epoch -  35
Epoch -  36
Epoch -  37
Epoch -  38
Epoch -  39
Epoch -  40
Epoch -  41
Epoch -  42
Epoch -  43
Epoch -  44
Epoch -  45
Epoch -  46
Epoch -  47
Epoch -  48
Epoch -  49
Epoch -  50
Epoch -  51
Epoch -  52
Epoch -  53
Epoch -  54
Epoch -  55
Epoch -  56
Epoch -  57
Epoch -  58
Epoch -  59
Epoch -  60
Epoch -  61
Epoch -  62
Epoch -  63
Epoch -  64
Epoch -  65
Epoch -  66
Epoch -  67
Epoch -  68
Epoch -  69
Epoch -  70
Epoch -  71
Epoch -  72
Epoch -  73
Epoch -  74
Epoch -  75
Epoch -  76
Epoch -  77
Epoch -  78
Epoch -  79
Epoch -  80
Epoch -  81
Epoch -  82
Epoch -  83
Ep

In [0]:
#ejemplo predicción
predic = sess.run(prediction, {data: [[[1], [0], [0], [1], [1], [0], [1], [1], [1], [0], [1], [0], [0], [1], [1], [0], [1], [1], [1], [0]]]})
output = np.argmax(predic)
print(output)

In [0]:
a = [[15], [11], [13], [8], [13], [11], [7], [5], [12], [9], [12], [7], [12], [8], [10], [10], [9], [10], [9], [12], [7], [10], [10], [8], [12], [8], [12], [12], [5], [11], [10], [10], [5], [9], [12], [8], [9], [14], [8], [11], [5], [8], [8], [12], [10], [6], [12], [7], [9], [8], [9], [9], [14], [15], [11], [12], [11], [7], [8], [11], [9], [14], [11], [14], [13], [11], [13], [10], [10], [9], [7], [12], [7], [11], [10], [9], [12], [11], [9], [8], [8], [12], [11], [5], [6], [11], [9], [15], [14], [7], [5], [8], [11], [10], [8], [10], [10], [12], [15], [7], [10], [9], [10], [9], [12], [10], [13], [11], [12], [7], [10], [9], [9], [10], [7], [13], [3], [9], [7], [10], [9], [11], [12], [11], [6], [10], [11], [9], [14], [9], [9], [9], [12], [10], [7], [7], [12], [7], [9], [10], [11], [13], [12], [11], [6], [8], [13], [11], [12], [8], [13], [11], [12], [13], [10], [15], [10], [11], [9], [8], [9], [8], [7], [14], [11], [9], [12], [10], [10], [10], [10], [14], [5], [14], [15], [15], [9], [10], [13], [9], [14], [10], [13], [10], [13], [11], [10], [14], [11], [10], [9], [11], [10], [11], [10], [6], [11], [12], [16], [9], [11], [9], [8], [12], [14], [10], [10], [7], [12], [10], [9], [8], [6], [7], [10], [11], [12], [6], [8], [10], [8], [6], [11], [10], [7], [8], [12], [9], [7], [10], [14], [10], [10], [12], [11], [9], [9], [8], [12], [14], [11], [9], [9], [10], [12], [8], [6], [8], [11], [7], [12], [11], [13], [9], [9], [10], [9], [11], [11], [6], [11], [9], [12], [9], [12], [12], [10], [7], [8], [8], [14], [11], [9], [8], [10], [11], [12], [8], [12], [9], [10], [10], [8], [11], [12], [12], [7], [9], [6], [6], [13], [12], [8], [9], [11], [12], [15], [9], [12], [8], [10], [9], [9], [13], [11], [7], [11], [14], [11], [12], [10], [2], [15], [8], [10], [13], [10], [10], [9], [9], [15], [9], [14], [9], [9], [16], [13], [11], [10], [7], [11], [11], [14], [10], [8], [9], [6], [10], [13], [8], [11], [11], [9], [10], [11], [8], [9], [9], [10], [11], [11], [5], [13], [8], [12], [11], [8], [10], [5], [7], [9], [12], [7], [7], [11], [12], [10], [8], [8], [11], [11], [9], [7], [13], [8], [10], [10], [13], [10], [10], [10], [11], [8], [10], [9], [10], [10], [10], [12], [10], [7], [10], [6], [9], [9], [8], [5], [12], [12], [10], [8], [8], [12], [14], [12], [11], [10], [10], [4], [11], [10], [9], [9], [12], [9], [14], [8], [11], [13], [11], [11], [10], [10], [9], [10], [12], [11], [11], [10], [9], [7], [12], [13], [10], [12], [11], [10], [9], [13], [13], [11], [10], [9], [14], [6], [4], [11], [4], [8], [9], [13], [10], [11], [12], [10], [11], [7], [9], [10], [4], [12], [10], [13], [8], [11], [7], [9], [11], [9], [12], [10], [12], [11], [12], [14], [12], [10], [11], [10], [9], [11], [6], [11], [7], [12], [8], [8], [12], [11], [8], [11], [15], [11], [9], [11], [13], [9], [14], [11], [13], [8], [7], [10], [12], [8], [12], [7], [11], [9], [10], [10], [7], [9], [12], [11], [11], [10], [8], [7], [10], [7], [8], [9], [8], [6], [11], [12], [13], [10], [7], [10], [7], [9], [8], [10], [7], [10], [11], [8], [6], [8], [12], [10], [13], [10], [11], [12], [10], [8], [12], [11], [10], [12], [7], [8], [13], [10], [9], [8], [5], [11], [10], [10], [7], [12], [11], [9], [8], [12], [11], [9], [13], [10], [9], [9], [8], [9], [9], [11], [10], [10], [10], [11], [9], [12], [12], [9], [10], [7], [9], [16], [4], [10], [10], [13], [14], [13], [6], [8], [12], [11], [12], [13], [10], [10], [12], [11], [11], [11], [14], [11], [8], [7], [9], [8], [9], [11], [10], [7], [6], [12], [11], [11], [10], [11], [12], [8], [10], [11], [12], [12], [11], [10], [12], [10], [9], [5], [5], [11], [8], [13], [15], [8], [4], [11], [11], [12], [6], [9], [13], [10], [12], [12], [12], [7], [7], [10], [11], [5], [9], [9], [11], [7], [13], [10], [11], [9], [12], [7], [15], [7], [11], [10], [9], [14], [7], [12], [10], [11], [14], [9], [9], [13], [12], [8], [12], [12], [5], [13], [9], [9], [10], [15], [6], [9], [7], [9], [6], [8], [13], [11], [14], [11], [10], [7], [9], [12], [8], [12], [12], [15], [13], [8], [9], [7], [8], [8], [7], [10], [13], [9], [5], [8], [10], [9], [9], [9], [8], [10], [12], [8], [12], [5], [11], [8], [10], [9], [14], [10], [11], [10], [8], [10], [6], [12], [15], [7], [11], [10], [7], [13], [9], [6], [11], [9], [13], [13], [11], [10], [10], [11], [8], [13], [13], [9], [12], [11], [11], [10], [12], [10], [11], [8], [9], [9], [7], [10], [8], [9], [8], [12], [10], [8], [9], [10], [10], [13], [9], [12], [9], [10], [12], [11], [15], [11], [9], [12], [6], [9], [10], [11], [12], [7], [9], [12], [10], [12], [8], [11], [11], [9], [10], [10], [7], [14], [7], [9], [11], [7], [11], [10], [13], [11], [7], [7], [16], [11], [8], [9], [13], [9], [12], [10], [10], [9], [8], [8], [10], [11], [12], [10], [10], [9], [9], [10], [7], [7], [7], [12], [13], [11], [6], [9], [13], [8], [8], [12], [9], [10], [11], [12], [11], [12], [12], [10], [13], [12], [10], [9], [9], [10], [10], [9], [9], [12], [13], [6], [10], [8], [11], [11], [9], [14], [9], [9], [9], [10], [11], [10], [12], [5], [12], [5], [10], [13], [11], [7], [9], [10], [9], [12], [11], [10], [11], [13], [11], [11], [11], [7], [9], [10], [10], [11], [11], [11], [12], [9], [11], [12], [12], [14], [13], [11], [13], [9], [8], [12], [13], [8], [7], [13], [9], [7], [7], [10], [11], [10], [12], [11], [9], [10], [11], [12], [11], [7], [14], [9], [12], [11], [12], [14], [9], [12], [9], [9], [12], [6], [9], [11], [11], [11], [10], [16], [10], [8], [12], [7], [8], [6], [11], [11], [12], [10], [11], [11], [11], [10], [6], [9], [9], [7], [12], [14], [12], [11], [11], [11], [11], [9], [12], [13], [13], [8], [11], [8], [6], [10], [10], [9], [8]]
salida = []
for e in a:
  salida.append([str(e[0])])
print(salida)

[['15'], ['11'], ['13'], ['8'], ['13'], ['11'], ['7'], ['5'], ['12'], ['9'], ['12'], ['7'], ['12'], ['8'], ['10'], ['10'], ['9'], ['10'], ['9'], ['12'], ['7'], ['10'], ['10'], ['8'], ['12'], ['8'], ['12'], ['12'], ['5'], ['11'], ['10'], ['10'], ['5'], ['9'], ['12'], ['8'], ['9'], ['14'], ['8'], ['11'], ['5'], ['8'], ['8'], ['12'], ['10'], ['6'], ['12'], ['7'], ['9'], ['8'], ['9'], ['9'], ['14'], ['15'], ['11'], ['12'], ['11'], ['7'], ['8'], ['11'], ['9'], ['14'], ['11'], ['14'], ['13'], ['11'], ['13'], ['10'], ['10'], ['9'], ['7'], ['12'], ['7'], ['11'], ['10'], ['9'], ['12'], ['11'], ['9'], ['8'], ['8'], ['12'], ['11'], ['5'], ['6'], ['11'], ['9'], ['15'], ['14'], ['7'], ['5'], ['8'], ['11'], ['10'], ['8'], ['10'], ['10'], ['12'], ['15'], ['7'], ['10'], ['9'], ['10'], ['9'], ['12'], ['10'], ['13'], ['11'], ['12'], ['7'], ['10'], ['9'], ['9'], ['10'], ['7'], ['13'], ['3'], ['9'], ['7'], ['10'], ['9'], ['11'], ['12'], ['11'], ['6'], ['10'], ['11'], ['9'], ['14'], ['9'], ['9'], ['9'], ['